# Lyfe - Une certaine définition du vivant

Projet informatique mené par le Grand Vass et le modeste Till dans le cadre de l'UE "Projet numérique" pendant l'année scolaire 2021-2022

# Définition du problème

## Introduction

Nous cherchons dans ce projet à reproduire l'emergence et le comportement de structure bien connus de la nature : les motifs de Turing. Ces motifs nous sont familiers, on les retrouve par exemple sur le poisson globe-géant, la fourrure d'un léopard ou encore sur certains coquillages. L'origine de ces motifs très réguliers n'est pas encore parfaitement comprise car elle repose sur des mécanismes biologique complexe : la morphogénèse, la différenciation, etc. Néanmoins, le mathématician Alan Turing proposa en 1952 dans son essai "The Chemical Basis of Morphogenesis" une première explication. Il suggère une modélisation thermochimique simple à ce phénomène. Sous ce prisme, il démontre que les motifs de Turing émergent naturellement comme solution des équations de réactions-diffusions. Nous allons tenter de résoudre ces équations pour visualiser ces motifs.

<img src="globe.png"  width="1000" />

## Modélisation Théorique Succinte

En réalité, les équations de diffusion-réactions sont une grande classe d'équations aux dérivées partielles et nous n'allons pas chercher à résoudre chacun d'elle. On retrouve parmi elle le système d'équation de Gray-Scott dont nous rappelons la construction :considérons 2 espèces chimique V et U (et une troisième espèce P qui est un déchet de réaction). Les équations chimique qui régissent leurs évolution sont les suivantes :

In [9]:
%%latex
\begin{align}
U + 2V & \rightarrow 3V \\
V & \rightarrow P \\
\end{align} 

<IPython.core.display.Latex object>

De ces équations nous déduisons l'évolution dans le temps et dans l'espace des espèces U et V, ce sont les équations de Gray-Scott.

In [8]:
%%latex
\begin{align}
\frac{\partial U}{\partial t} & = D_{u}\Delta U - UV^{2} + F(1-U)\\
\frac{\partial V}{\partial t} & = D_{v}\Delta V + UV^{2} - (F+k)V\\
\end{align} 

<IPython.core.display.Latex object>

Sans rentrer dans la construction de ces équations, nous pouvons identifier 2 phénomènes couplés dans ces équations. D'une part les phénomènes de diffusion thermique reconnaissable par la présence de l'opérateur Laplacien $\Delta$ et des coefficients de diffusion $D_{u}$ et $D_{v}$. D'autre part des phénomènes de cinétique chimique caractérisés par le coefficients d'annihilation $k$ et de création $F$.

## Objectif du projet

Ces équations, de part leurs structures, n'ont pas de solution analytique. L'emploi d'une résolution numérique semble alors justifiée. Le chercheur John E. Pearson, du laboratoire national de Los Alamos, proposa dans un article qui fait désormais référence, des résolutions de ces équations pour différents paramètres $F$ et $k$. Il dresse notamment un "diagramme de phase" qui repertorie quels motifs régulier émergent en fonction de ces paramètres. Concrètement, notre objectif sera d'obtenir les mêmes résultats. Nous chercherons dans un second temps à analyser tout ces motifs pour essayer de trouver des comportements généraux caractéristiques. En d'autres termes, notre rapport se concentrera dans un premier temps sur l'implémentation de la résolution numérique et dans un second temps de l'analyse physique de nos simulations.

# Exécution du projet

## Outils utilisés

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from PyQt5 import QtWidgets
import time
import datetime
import os
import scipy.optimize as opt

Notre projet se portait très bien à une résolution par approche procédurale. Notre code appelera à tour de rôle les fonctions qui lui permettront de calculer et d'afficher les résultats souhaités. Nous utilisons l'arsenal usuel de librairies python:
- Numpy et scipy.optimize pour les calculs :
- Matplotlib et PyQt5 pour l'affichage et l'animation
- Time, datetime et os pour l'organisation des données récoltées

Explicitions désormais les différentes fonctions que nous avons utilisés.

## La fonction opérateur laplacien $\Delta$

In [5]:
def lap(M, dx, dy):
    """Permet d'obtenir le laplacien discret de la matrice M"""
    L = - (2*M)/(dx**2) - (2*M)/(dy**2)
    L += (np.roll(M, (0,-1), (0,1)))/(dx**2) # voisin de droite
    L += (np.roll(M, (0,+1), (0,1)))/(dx**2) # voisin de gauche
    L += (np.roll(M, (-1,0), (0,1)))/(dy**2) # voisin du dessus
    L += (np.roll(M, (+1,0), (0,1)))/(dy**2) # voisin du dessous
    return L


Le Laplacien est un opérateur différentiel d'ordre 2. Nous avons l'avons implementé à partir de sa définition par la méthode des différences finies. La fonction va dans un premier temps retrancher dans les deux directions par la résolution spatiale. A chaque appel, le laplacien sera calculé en repoussant les bords de la matrice grâce aux fonctions $\texttt{np.roll}$. On évite ainsi les effets de bors.
Nous avions tenté une première implémentation du laplacian en utilisant des combinaisons de $\texttt{np.grad}$ et en reconstruisant l'identité $ \Delta = \overrightarrow{\nabla} \cdot \overrightarrow{\nabla} $
Pour le reste, je suis en train de décortiquer internet, mais je suis pas certain de trouver une réponse convaincante...

## Les équations de Gray-Scott


In [6]:
def G_S(u, v, f, k, Du, Dv, dx, dy):
    """Renvoie les valeurs des derivees temporelles des concentrations en U et␣
    ,→V selon le modele de reaction-diffusion de Gray-Scott"""
    """u et v sont des matrices contenant les concentrations en U et V (en mol.
    ,→L¹), f est le "feed-rate"(en s¹), k (en s¹) est le taux de conversion de U␣
    ,→(en P, inerte), et Du (resp. Dv) est le coefficient de diffusion de U (resp.␣
    ,→V) (en m².s¹)."""
    u_dot = Du * lap(u, dx, dy) - u * v**2 + f * (1-u)
    v_dot = Dv * lap(v, dx, dy) + u * v**2 - (f + k) * v
    return (u_dot, v_dot)

Il s'agit de la fonction centrale de notre programme puisqu'elle implémente les équations de gray-scott présentées en première partie de ce rapport. Il ne s'agit en rien d'autre que les expressions mathématiques de ces dernières. Cette fonction sera ensuite appelée dans un schéma de résolution de type Euler.